# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
sma_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                sma_evaluation_results['image_name'].append(image_name)
                sma_evaluation_results['thresholds'].append(threshold)
                sma_evaluation_results['fitness_function'].append(fitness_function)
                sma_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    sma = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    slimeMoulds, best_thresholds = sma.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(sma.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(sma.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                sma_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                sma_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                sma_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                sma_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                sma_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                sma_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                sma_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                sma_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                sma_evaluation_results['Regions'].append(mean_regions)
                sma_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                sma_evaluation_results['Fitness'].append(fitness_values)
                sma_evaluation_results['MSE'].append(list_mse)
                sma_evaluation_results['RMSE'].append(list_rmse)
                sma_evaluation_results['PSNR'].append(list_psnr)
                sma_evaluation_results['SSIM'].append(list_ssim)
                sma_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1926.269924222515
1907.4489660376119
1906.377195180803
1917.8407517007674
1917.5726853745625
1929.851401043185
1946.809716789367
1887.1666744733486
1937.7891065623767
1934.5683046202053
1927.249730217372
1923.0672366430108
1897.7971612650067
1878.1343727239173
1921.6493572255974
1940.0039749597672
1890.452258326075
1886.2841572160082
1938.1132289545483
1947.4851111166436
1924.2304940983365
1912.0999829358202
1907.9481507578266
1947.1797867035743
1905.7624909887104
1926.3937954316889
1888.3722116601293
1867.6029259221107
1885.3869596272439
1934.7218091372802
Mean Fitness = 1915.3876640638466
	 AirplaneF16.tiff | Threshold =  3
2002.5089768687294
1997.8706206717206
2000.252306943221
2013.5979344106581
1976.5572855334574
1977.6615552645983
2003.9307600108577
1989.8281598466735
2001.0622952971191
1954.5862726654461
1970.9044031493677
1937.1117116797277
1900.0308849394476
1977.072007584185
1995.9008185341509
1953.9125478710791
1937.0660371895449
1959.6271

3140.721630053015
3055.677985658495
Mean Fitness = 3087.7964555320063
	 Mandrill.tiff | Threshold =  2
1493.1096692329454
1509.1647209637229
1503.0394253029588
1527.2876045161556
1514.0519634008801
1222.2582112006162
1349.9108632539392
1493.9053726368406
1393.761906699209
1257.4538644751915
1493.9053726368406
1482.9742299636162
1534.3151528317214
1371.403218189681
1375.198640508364
1386.077323632292
1532.021494306653
1328.01905547242
1260.01683604841
1486.675190903261
1511.964333144204
1464.9908331447095
1523.9831682743752
1406.5815330971004
1289.2886244374838
1215.0283940610116
1446.8338780939011
1548.3075195992974
1469.384178064887
1517.4772600667675
Mean Fitness = 1430.2796612719821
	 Mandrill.tiff | Threshold =  3
1601.8442932456767
1531.6860551758373
1562.7351244347635
1506.0780634769294
1502.6894414119001
1506.9796026026065
1391.4786861687262
1515.6140294557476
1631.920391758787
1600.926722129796
1392.625247421546
1478.3284156258103
1521.756782246942
1518.2033141243298
1566.66541

4162.549765810045
4141.081589739516
4133.88502226936
4186.029643429935
4186.879459064089
4134.018073597874
4058.4452363555015
Mean Fitness = 4128.989981362192


## Visualize Results using DataFrame

In [7]:
sma_evaluation_results_df =pd.DataFrame(sma_evaluation_results)
sma_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[116, 174]",1.647149,1915.387664,2356.861719,48.474336,14.434384,0.769118,0.935706,"[[116, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.227388620376587, 1.2775685787200928, 1.2800...","[1926.269924222515, 1907.4489660376119, 1906.3...","[2352.882354736328, 2627.110153198242, 2484.58...","[48.50651868291857, 51.25534267955139, 49.8455...","[14.4148014808464, 13.936020779854497, 14.1782...","[0.7874132261288455, 0.7654830036465561, 0.775...","[0.9362953899311315, 0.9337950028074978, 0.935..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[82, 139, 185]",1.750569,1973.501423,1844.958694,42.412338,15.707737,0.765746,0.950014,"[[82, 255, 255, 255, 185, 255, 255, 255, 255, ...","[1.726719617843628, 1.367664098739624, 1.42386...","[2002.5089768687294, 1997.8706206717206, 2000....","[2153.674819946289, 2077.269344329834, 1243.06...","[46.40770216188568, 45.577070378972735, 35.257...","[14.79900230412914, 14.95587548891203, 17.1858...","[0.8004378201527759, 0.8140695499120058, 0.698...","[0.946532177055819, 0.9481431252399463, 0.9621..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[57, 108, 153, 191]",1.916326,2006.876896,1658.981838,40.089042,16.237250,0.769864,0.955230,"[[108, 255, 255, 191, 191, 191, 255, 191, 255,...","[2.0733585357666016, 1.884068489074707, 2.2372...","[2012.6159813283493, 2018.1260999028718, 2026....","[1178.8403396606445, 1766.3195991516113, 1707....","[34.33424441662645, 42.027605203623146, 41.320...","[17.41625371972799, 15.660110729571032, 15.807...","[0.7044616569572991, 0.7770086042769059, 0.735...","[0.9625129298424179, 0.9526125815726354, 0.950..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[43, 103, 144, 175, 202]",1.672301,2034.592196,1219.962675,33.809203,17.863413,0.774904,0.964790,"[[103, 202, 202, 202, 202, 202, 202, 202, 202,...","[1.6098580360412598, 1.8194622993469238, 1.320...","[2021.1397651881025, 2019.3450044680562, 2046....","[523.5588531494141, 2004.0127906799316, 811.65...","[22.881408460787856, 44.76620143232986, 28.489...","[20.941148531502705, 15.11179871764525, 19.037...","[0.8047669133519041, 0.8096809020126404, 0.736...","[0.9746346642564884, 0.9516165736973813, 0.971..."
4,Lena.png,2,mMasi_entropy,max,"[93, 157]",1.636681,1884.465972,2945.228950,53.303259,13.733463,0.605682,0.872509,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.636430263519287, 1.6489286422729492, 1.6254...","[1956.079275838022, 1950.0699958143555, 1805.7...","[2108.880683898926, 3216.412887573242, 6008.52...","[45.92255092978749, 56.713427753692, 77.514647...","[14.890283519156018, 13.057085672824835, 10.34...","[0.6173784072748878, 0.594936239212573, 0.5659...","[0.8914785403440494, 0.8742462732322591, 0.817..."
5,Lena.png,3,mMasi_entropy,max,"[77, 131, 179]",1.663594,1996.688739,2032.123204,44.076579,15.421084,0.647540,0.904952,"[[179, 179, 179, 179, 179, 179, 179, 179, 179,...","[1.6244158744812012, 1.6709518432617188, 1.696...","[1971.5616465144017, 1978.2833686078168, 1936....","[2035.7450065612793, 1382.7996788024902, 1940....","[45.11923100587242, 37.18601455927336, 44.0484...","[15.043569826740866, 16.723210908757032, 15.25...","[0.647432935767496, 0.651929901718521, 0.61669...","[0.9035908622160946, 0.9201224073860068, 0.893..."
6,Lena.png,4,mMasi_entropy,max,"[45, 96, 140, 187]",1.802286,2067.695876,1528.815554,38.279266,16.642886,0.685039,0.923629,"[[187, 187, 187, 187, 187, 187, 187, 187, 187,...","[1.4598894119262695, 2.0580685138702393, 1.761...","[1972.5194543766652, 2118.3341219875538, 2084....","[3830.602523803711, 1114.7202987670898, 1180.9...","[61.891861531252324, 33.38742725588616, 34.364...","[12.298132704025384, 17.65914451277889, 17.408...","[0.6428446568064026, 0.7142786397810285, 0.674...","[0.8914007849321718, 0.9313888090296961, 0.925..."
7,Len

# Save results

In [8]:
pickle.dump(sma_evaluation_results_df, open('results/evaluation/SMA/SMA_masi_df.pkl', 'wb'))
pickle.dump(sma_evaluation_results, open('results/evaluation/SMA/SMA_masi_dict.pkl', 'wb'))